In [1]:
from urllib import request
import pandas as pd
from matplotlib import pyplot as plt
import json
#import mpld3
from datetime import date
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
from matplotlib import colors
import numpy as np
import datetime as dt
#import gdal as gdal
from shapely.geometry import Polygon
from shapely import wkt
from itertools import chain
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

#response = request.urlopen(l1)
#dat = response.read()
#data = json.loads(dat)
#df = pd.json_normalize(data['features'])

In [2]:
map1=gpd.read_file("parcel_map_tax_bills.rar/parcel_map_tax_bills.shp")

In [3]:
map1=map1[~map1.lu_descrip.str.contains("Ex-",na=False)]

In [4]:
comm_indust_otherbusiness=["office_bldgs_not_in_ind_park","commercial",'in_industrial_park',
'main_street','not_in_industrial_park','shopping_centers',
'office_bldgs_in_ind_park','industrial_buildings',]

# Total Value Top 7 Townships

In [35]:
map1[(map1.municipali.str.contains("Doylestown B"))&(map1.code_class=='residential')].munibill22.mean()

544.5217303284951

In [5]:
#pd.set_option('display.float_format', '${:,.2f}'.format)
map1[map1.municipali.str.contains(" Township")].groupby('municipali').TOTAL_VALU.sum().sort_values(ascending=False).head(10)

municipali
Bensalem Township           653366500.0
Northampton Township        599098240.0
Middletown Township         564012370.0
Lower Makefield Township    552408190.0
Bristol Township            431529380.0
Buckingham Township         399094510.0
Warrington Township         377617490.0
Newtown Township            352965230.0
Warminster Township         349454050.0
Falls Township              337434040.0
Name: TOTAL_VALU, dtype: float64

In [41]:
map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].groupby('municipali').munibill22.mean().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Average Municipal Tax Bill'})

,municipali,munibill22
0,Bensalem Township,508.80
1,Bristol Township,439.86
2,Buckingham Township,217.05
3,Lower Makefield Township,868.31
4,Middletown Township,521.03
5,Northampton Township,869.12
6,Warrington Township,609.78


In [43]:
top_values=map1[map1.municipali.str.contains(" Township")].groupby('municipali').TOTAL_VALU.sum().round(2).sort_values(ascending=False).head(7).to_frame().reset_index()
ave_res_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].groupby('municipali').ttl_bill22.mean().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Average Residential Tax Bill'})
ave_muni_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].groupby('municipali').munibill22.mean().round(2).to_frame().reset_index().rename(columns={'munibill22':'Average Municipal Tax Bill'})
res_counts=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].municipali.value_counts().to_frame().rename(columns={'municipali':'total residential properties'}).reset_index().rename(columns={'index':'municipali'})
bus_counts=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].municipali.value_counts().to_frame().rename(columns={'municipali':'total business properties'}).reset_index().rename(columns={'index':'municipali'})
ave_bus_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].groupby('municipali').ttl_bill22.mean().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Average Business Tax Bill'})
median_bus_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].groupby('municipali').ttl_bill22.median().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Median Business Tax Bill'})
top_values=pd.merge(top_values,ave_res_bills)
top_values=pd.merge(top_values,res_counts)
top_values=pd.merge(top_values,bus_counts)
top_values=pd.merge(top_values,median_bus_bills)
top_values=pd.merge(top_values,ave_bus_bills)
top_values=pd.merge(top_values,ave_muni_bills)
#top_values['ttl_v_rank']=top_values.index+1
#top_values['mrtx_rank']=top_values.sort_values(by="Average Residential Tax Bill").index+1
#top_values['trp_rank']=top_values.sort_values(by="total residential properties").index+1
#top_values['tbp_rank']=top_values.sort_values(by="total business properties").index+1
#top_values['medbtx_rank']=top_values.sort_values(by="Median Business Tax Bill").index+1
#top_values['avebtx_rank']=top_values.sort_values(by="Average Business Tax Bill").index+1
township_stats=top_values
township_stats['ARTB_obj']=township_stats['Average Residential Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
township_stats['MBTB_obj']=township_stats['Median Business Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
township_stats['ABTB_obj']=township_stats['Average Business Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
township_stats['Total Tax Assessed Land Value']=township_stats['TOTAL_VALU'].apply(lambda x: "${:,.2f}".format((x/1)))
township_stats['AMB']=township_stats['Average Municipal Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))


In [44]:
township_stats

,municipali,TOTAL_VALU,Average Residential Tax Bill,total residential properties,total business properties,Median Business Tax Bill,Average Business Tax Bill,Average Municipal Tax Bill,ARTB_obj,MBTB_obj,ABTB_obj,Total Tax Assessed Land Value,AMB
0,Bensalem Township,653366500.0,5234.44,15096,886,15851.30,49229.77,508.80,"$5,234.44","$15,851.30","$49,229.77","$653,366,500.00",$508.80
1,Northampton Township,599098240.0,7233.62,13852,305,11446.82,19088.24,869.12,"$7,233.62","$11,446.82","$19,088.24","$599,098,240.00",$869.12
2,Middletown Township,564012370.0,6186.56,13640,544,9136.52,35805.54,521.03,"$6,186.56","$9,136.52","$35,805.54","$564,012,370.00",$521.03
3,Lower Makefield Township,552408190.0,9315.33,11856,305,5456.82,23886.97,868.31,"$9,315.33","$5,456.82","$23,886.97","$552,408,190.00",$868.31
4,Bristol Township,431529380.0,4944.68,16640,712,14853.30,34830.65,439.86,"$4,944.68","$14,853.30","$34,830.65","$431,529,380.00",$439.86
5,Buckingham Township,399094510.0,8432.97,6761,196,8547.55,17563.32,217.05,"$8,432.97","$8,547.55","$17,563.32","$399,094,510.00",$217.05
6,Warrington Township,377617490.0,6337.27,8326,193,16739.60,44036.91,609.78,"$6,337.27","$16,739.60","$44,036.91","$377,617,490.00",$609.78


In [48]:
top_values=map1[map1.municipali.str.contains(" Borough")].groupby('municipali').TOTAL_VALU.sum().round(2).sort_values(ascending=False).head(7).to_frame().reset_index()
ave_res_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].groupby('municipali').ttl_bill22.mean().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Average Residential Tax Bill'})
ave_muni_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].groupby('municipali').munibill22.mean().round(2).to_frame().reset_index().rename(columns={'munibill22':'Average Municipal Tax Bill'})
res_counts=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class=='residential')].municipali.value_counts().to_frame().rename(columns={'municipali':'total residential properties'}).reset_index().rename(columns={'index':'municipali'})
bus_counts=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].municipali.value_counts().to_frame().rename(columns={'municipali':'total business properties'}).reset_index().rename(columns={'index':'municipali'})
ave_bus_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].groupby('municipali').ttl_bill22.mean().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Average Business Tax Bill'})
median_bus_bills=map1[(map1.municipali.isin(top_values.municipali))&(map1.code_class.isin(comm_indust_otherbusiness))].groupby('municipali').ttl_bill22.median().round(2).to_frame().reset_index().rename(columns={'ttl_bill22':'Median Business Tax Bill'})
top_values=pd.merge(top_values,ave_res_bills)
top_values=pd.merge(top_values,res_counts)
top_values=pd.merge(top_values,bus_counts)
top_values=pd.merge(top_values,median_bus_bills)
top_values=pd.merge(top_values,ave_bus_bills)
top_values=pd.merge(top_values,ave_muni_bills)
#top_values['ttl_v_rank']=top_values.index+1
#top_values['mrtx_rank']=top_values.sort_values(by="Average Residential Tax Bill").index+1
#top_values['trp_rank']=top_values.sort_values(by="total residential properties").index+1
#top_values['tbp_rank']=top_values.sort_values(by="total business properties").index+1
#top_values['medbtx_rank']=top_values.sort_values(by="Median Business Tax Bill").index+1
#top_values['avebtx_rank']=top_values.sort_values(by="Average Business Tax Bill").index+1
borough_stats=top_values
borough_stats['ARTB_obj']=borough_stats['Average Residential Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
borough_stats['MBTB_obj']=borough_stats['Median Business Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
borough_stats['ABTB_obj']=borough_stats['Average Business Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))
borough_stats['Total Tax Assessed Land Value']=borough_stats['TOTAL_VALU'].apply(lambda x: "${:,.2f}".format((x/1)))
borough_stats['AMB']=borough_stats['Average Municipal Tax Bill'].apply(lambda x: "${:,.2f}".format((x/1)))

In [49]:
borough_stats

,municipali,TOTAL_VALU,Average Residential Tax Bill,total residential properties,total business properties,Median Business Tax Bill,Average Business Tax Bill,Average Municipal Tax Bill,ARTB_obj,MBTB_obj,ABTB_obj,Total Tax Assessed Land Value,AMB
0,Doylestown Borough,119873280.0,5165.94,2618,329,9132.73,14982.42,544.52,"$5,165.94","$9,132.73","$14,982.42","$119,873,280.00",$544.52
1,Perkasie Borough,91954880.0,4707.33,2923,117,6376.73,10956.99,256.20,"$4,707.33","$6,376.73","$10,956.99","$91,954,880.00",$256.20
2,Quakertown Borough,84446830.0,3929.81,2625,249,9325.08,21549.60,32.60,"$3,929.81","$9,325.08","$21,549.60","$84,446,830.00",$32.60
3,Bristol Borough,81580360.0,4165.73,3028,315,6198.05,15572.28,927.38,"$4,165.73","$6,198.05","$15,572.28","$81,580,360.00",$927.38
4,New Hope Borough,64769630.0,7194.59,1043,144,7992.75,12260.21,672.24,"$7,194.59","$7,992.75","$12,260.21","$64,769,630.00",$672.24
5,Morrisville Borough,62376750.0,5644.63,2727,139,9569.72,16900.80,985.93,"$5,644.63","$9,569.72","$16,900.80","$62,376,750.00",$985.93
6,Chalfont Borough,54677590.0,5274.95,1566,53,8550.86,13593.77,567.42,"$5,274.95","$8,550.86","$13,593.77","$54,677,590.00",$567.42


In [51]:
township_stats.to_clipboard()

In [52]:
borough_stats.to_clipboard()